In [7]:
import supervision as sv
import torch, torchvision
import glob
from mmdet.apis import init_detector, inference_detector
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

model_folder="faster_rcnn"
base_model_name="faster-rcnn_r50_fpn_1x_coco"
custom_model_name=f"custom-{base_model_name}-lab"
CUSTOM_CONFIG_PATH = f"{HOME}/mmdetection/configs/{model_folder}/{custom_model_name}.py"
dataset_location=f"/work/van-speech-nlp/jindaznb/j-vis/Forestfire2024-1-lab"
print(CUSTOM_CONFIG_PATH)

seed_value= 42 

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value

os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)
# dataset_location=f"/work/van-speech-nlp/jindaznb/j-vis/Forestfire2024-1"
dataset_location=f"/work/van-speech-nlp/jindaznb/j-vis/Forestfire2024-1-lab"

# # 4. Set `tensorflow` pseudo-random generator at a fixed value
# import tensorflow as tf
# tf.random.set_seed(seed_value)


param_space = {
    'CONFIDENCE_THRESHOLD': np.linspace(0.1, 1.0, 20),  # Adjust the range as needed
    'NMS_IOU_THRESHOLD': np.linspace(0.1, 1.0, 20),  # Adjust the range as needed
}

best_map = 0.0
best_params = {}

MAX_EPOCHS=300
BATCH_SIZE=8
classes=('Alive Tree', 'Beetle-Fire Tree', 'Dead Tree', 'Debris')
num_classes=len(classes)


from bayes_opt import BayesianOptimization

def optimize_params(ds, model, param_space, num_samples=30):
    def objective_function(confidence_threshold, nms_iou_threshold):
        def callback(image: np.ndarray) -> sv.Detections:
            result = inference_detector(model, image)
            detections = sv.Detections.from_mmdetection(result)
            return detections[detections.confidence > confidence_threshold].with_nms(threshold=nms_iou_threshold)

        mean_average_precision = sv.MeanAveragePrecision.benchmark(dataset=ds, callback=callback)
        return mean_average_precision.map50_95

    optimizer = BayesianOptimization(
        f=objective_function,
        pbounds={'confidence_threshold': (param_space['CONFIDENCE_THRESHOLD'][0], param_space['CONFIDENCE_THRESHOLD'][-1]),
                 'nms_iou_threshold': (param_space['NMS_IOU_THRESHOLD'][0], param_space['NMS_IOU_THRESHOLD'][-1])},
        random_state=42,  
    )

    optimizer.maximize(init_points=5, n_iter=num_samples - 5)  

    best_params = [optimizer.max['params']['confidence_threshold'], optimizer.max['params']['nms_iou_threshold']]
    best_map = optimizer.max['target']

    return best_map, best_params


def calculate_mAP_default(dataset_location: str, model_name: str, custom_config_path: str, device: str = 'cuda', confidence_threshold: float = 0.5, nms_iou_threshold: float = 0.5) -> float:
    ds = sv.DetectionDataset.from_coco(
        images_directory_path=f"{dataset_location}/test",
        annotations_path=f"{dataset_location}/test/_annotations.coco.json",
    )

    images = list(ds.images.values())
    custom_weights_path = glob.glob(f"{HOME}/mmdetection/work_dirs/{model_name}/best_coco_bbox_mAP_50_epoch_*.pth")[-1]
    model = init_detector(custom_config_path, custom_weights_path, device=device)

    def callback(image: np.ndarray) -> sv.Detections:
        result = inference_detector(model, image)
        detections = sv.Detections.from_mmdetection(result)
        return detections[detections.confidence > confidence_threshold].with_nms(threshold=nms_iou_threshold)

    mean_average_precision = sv.MeanAveragePrecision.benchmark(
        dataset=ds,
        callback=callback
    )

    return mean_average_precision.map50_95

HOME: /work/van-speech-nlp/jindaznb/j-vis
/work/van-speech-nlp/jindaznb/j-vis/mmdetection/configs/faster_rcnn/custom-faster-rcnn_r50_fpn_1x_coco-lab.py


In [8]:
CONFIDENCE_THRESHOLD = 0.5
NMS_IOU_THRESHOLD = 0.5

ds = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset_location}/test",
    annotations_path=f"{dataset_location}/test/_annotations.coco.json",
)

images = list(ds.images.values())
CUSTOM_WEIGHTS_PATH = glob.glob(f"{HOME}/mmdetection/work_dirs/{custom_model_name}/best_coco_bbox_mAP_50_epoch_*.pth")[-1]
model = init_detector(CUSTOM_CONFIG_PATH, CUSTOM_WEIGHTS_PATH, device=DEVICE)

def callback(image: np.ndarray) -> sv.Detections:
    result = inference_detector(model, image)
    detections = sv.Detections.from_mmdetection(result)
    return detections[detections.confidence > CONFIDENCE_THRESHOLD].with_nms(threshold=NMS_IOU_THRESHOLD)


mean_average_precision = sv.MeanAveragePrecision.benchmark(
    dataset = ds,
    callback = callback
)

print('mAP:', mean_average_precision.map50_95)

Loads checkpoint by local backend from path: /work/van-speech-nlp/jindaznb/j-vis/mmdetection/work_dirs/custom-faster-rcnn_r50_fpn_1x_coco-lab/best_coco_bbox_mAP_50_epoch_297.pth
mAP: 0.32496726561590006


In [9]:
ds = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset_location}/test",
    annotations_path=f"{dataset_location}/test/_annotations.coco.json",
)

images = list(ds.images.values())
CUSTOM_WEIGHTS_PATH = glob.glob(f"{HOME}/mmdetection/work_dirs/{custom_model_name}/best_coco_bbox_mAP_50_epoch_*.pth")[-1]
model = init_detector(CUSTOM_CONFIG_PATH, CUSTOM_WEIGHTS_PATH, device=DEVICE)

best_map, best_params = optimize_params(ds, model, param_space)
print('Best mAP:', best_map)
print('Best Parameters:', best_params)

Loads checkpoint by local backend from path: /work/van-speech-nlp/jindaznb/j-vis/mmdetection/work_dirs/custom-faster-rcnn_r50_fpn_1x_coco-lab/best_coco_bbox_mAP_50_epoch_297.pth
|   iter    |  target   | confid... | nms_io... |
-------------------------------------------------
| 1         | 0.3189    | 0.4371    | 0.9556    |
| 2         | 0.3576    | 0.7588    | 0.6388    |
| 3         | 0.3189    | 0.2404    | 0.2404    |
| 4         | 0.3005    | 0.1523    | 0.8796    |
| 5         | 0.3446    | 0.641     | 0.7373    |
| 6         | 0.0       | 1.0       | 0.2199    |
| 7         | 0.4277    | 0.9904    | 1.0       |
| 8         | 0.45      | 0.9998    | 0.7859    |
| 9         | 0.399     | 0.9804    | 0.8087    |
| 10        | 0.435     | 0.9995    | 0.6693    |
| 11        | 0.3612    | 0.796     | 1.0       |
| 12        | 0.325     | 0.4976    | 0.8419    |
| 13        | 0.3285    | 0.5078    | 0.4724    |
| 14        | 0.2982    | 0.1       | 0.5284    |
| 15        | 0.342   